In [1]:
from ipywidgets import IntProgress, HTML, VBox, Button, HBox
from IPython.display import display
from time import sleep
import itertools

In [2]:
class ControlWidget:
    def __init__(self, sequence, size=None, every=None, name='Events'):
        self.__button = Button(description='Run', icon='play')
        self.__progress = IntProgress(min=0, max=1, value=0)
        self.__label = HTML()
        self.__box = HBox(children=[self.__button, self.__progress, self.__label])
        self.__button.on_click(self.__on_click)
        self.__every = every
        self.__is_size_unknown = False
        self.name = name
        self.__size = size
        self.set_new_sequence(sequence, size, every, name)
        
    def __on_click(self, b):
        self.__sequence, seq = itertools.tee(self.__sequence)
        index = 0
        try:
            for index, record in enumerate(seq, 1):
                if index == 1 or index % self.__every == 0:
                    if self.__is_size_unknown:
                        self.__progress.value = 1
                        self.__label.value = '{name}: {index} / ?'.format(
                            name=self.name,
                            index=index
                        )
                    else:
                        self.__progress.value = index
                        self.__label.value = u'{name}: {index} / {size}'.format(
                            name=self.name,
                            index=index,
                            size=self.__size
                        )
                #yield record
        except:
            self.__progress.bar_style = 'danger'
            raise
        else:
            self.__progress.bar_style = 'success'
            self.__progress.value = index
            self.__label.value = "{name}: {index}".format(
                name=self.name,
                index=str(index or '?')
            )
            
    def set_new_sequence(self, seq, size=None, every=None, name='Events'):
        self.__sequence = seq
        self.name = name
        
        self.__is_size_unknown = False
        self.__size = size
        self.__every = every
        if size is None:
            try:
                self.__size = len(seq)
            except TypeError:
                self.__is_size_unknown = True
        if size is not None:
            if every is None:
                if size <= 200:
                    self.__every = 1
                else:
                    self.__every = int(size / 200)
        else:
            if every is None:
                self.__every = 1

        if self.__is_size_unknown:
            self.__progress.min = 0
            self.__progress.max = 1
            self.__progress.value = 0
            self.__progress.bar_style = 'info'
        else:
            self.__progress.min = 0
            self.__progress.max = size
            self.__progress.value = 0
            self.__progress.bar_style = ''
        
    def draw(self):
        display(self.__box)

In [3]:
def gen(max):
    num = 0
    while num < max:
        # instead of sleep, extract event number here
        sleep(0.03)
        yield num
        num += 1

In [4]:
g = gen(100)
c = ControlWidget(g)
c.draw()

In [5]:
# set new sequence to run it again 
c.set_new_sequence(gen(150), 150, 5)